In [1]:
example = """1,0,1~1,2,1
0,0,2~2,0,2
0,2,3~2,2,3
0,0,4~0,2,4
2,0,5~2,2,5
0,1,6~2,1,6
1,1,8~1,1,9"""

In [2]:
with open('./data/Day 22/input.txt') as f:
    data = f.read()

In [3]:
def parse_information(inputstr):
    output = []
    max_x = max_y = max_z = 0
    for nr, line in enumerate(inputstr.splitlines()):
        x, y, z = line.split('~')[1].split(',')
        max_x = max(max_x, int(x))
        max_y = max(max_y, int(y))
        max_z = max(max_z, int(z))
        temp = list(tuple(int(i) for i in j.split(',')) for j in line.split('~'))
        temp.append(nr)
        output.append(temp)
    return sorted(output, key=lambda x: x[0][2]), max_x, max_y, max_z

In [4]:
def check_fall(x, y, z, x_end, y_end, data):
    out = []
    for i in range(x, x_end+1):
       out.append(data[(i, y, z-1)] == -1)

    for j in range(y, y_end+1):
       out.append(data[(x, j, z-1)] == -1)
    
    return all(out) 

In [5]:
import numpy as np

def fall_all_blocks(block_locations, max_x, max_y, max_z):
    supports = {}
    grid = -1*np.ones((max_x+1, max_y+1, max_z+1), dtype=int)
    grid[:,:,0] = -9999

    for block in block_locations:
        supports[block[2]] = set()
        z= block[0][2]
        z_top = block[1][2]

        while check_fall(block[0][0], block[0][1], z, block[1][0], block[1][1], grid):
            z -= 1
            z_top -= 1
        
        for x in range(block[0][0], block[1][0]+1):
            grid[x, block[0][1], z] = block[2]
            if grid[x, block[0][1], z-1] != -1:
                supports[block[2]].add(grid[x, block[0][1], z-1])
        
        for y in range(block[0][1], block[1][1]+1):
            grid[block[0][0], y, z] = block[2]
            if grid[block[0][0], y, z-1] != -1:
                supports[block[2]].add(grid[block[0][0], y, z-1])
        
        for k in range(z, z_top+1):
            grid[block[0][0], block[0][1], k] = block[2]
        
        if grid[block[0][0], block[0][1], z-1] != -1:
            supports[block[2]].add(grid[block[0][0], block[0][1], z-1])
    
    return grid, supports

In [6]:
def find_nr_removable_blocks(supports, grid):
    non_removable_blocks = set()
    for block, support in supports.items():
        if len(support) == 1:
            non_removable_blocks = non_removable_blocks.union(support)
    return len(np.unique(grid)) - 1 - len(non_removable_blocks)

# Part 1

In [7]:
ex_grid, support = fall_all_blocks(*parse_information(example))
find_nr_removable_blocks(support, ex_grid)

5

In [22]:
grid, support = fall_all_blocks(*parse_information(data))
find_nr_removable_blocks(support, grid)

517

# Part 2

In [26]:
from functools import lru_cache


def falling_blocks(start, isfalling):
    nr_falling = 0
    for supported_block, supporting_blocks in support.items():
        if supported_block != start and start in supporting_blocks:
            if all([blocki in isfalling for blocki in supporting_blocks]):
                isfalling.append(supported_block)
                nr_falling += 1 + falling_blocks(supported_block, isfalling)
    return nr_falling

In [27]:
nr_falls = {}
for key in support:
    nr_falls[key] = falling_blocks(key, [key])

In [28]:
sum(nr_falls.values())

61276